# RAG Vector Index Construction (Evaluate)

## Load Model

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
from tqdm import tqdm
import numpy as np
import json
from pathlib import Path

model_path = "../Models/bge-large-zh-v1.5"

embedder = SentenceTransformer(model_path)

## Read Corpus

In [4]:
data_file = "./data/llm_train.json"

def load_corpus(data_file: str):
    corpus = []

    with open(data_file, "r", encoding="utf-8") as f:
        datas  = json.load(f)
        for item in datas:
            corpus.append({
                "character": item["character"],
                "text": item["output"]
            })
    return pd.DataFrame(corpus)

df_corpus = load_corpus(data_file)
print("语料数量:", len(df_corpus))

语料数量: 5786


## Vectorization and Index Construction (FAISS)

In [5]:
# 按角色分组构建索引
character_indices = {}
character_dfs = {}

grouped = df_corpus.groupby("character")

for char_name, group in grouped:
    print(f"Processing character: {char_name} (Count: {len(group)})")
    
    # 编码
    embeddings = embedder.encode(
        group["text"].tolist(),
        show_progress_bar=True,
        batch_size=32,
        normalize_embeddings=True
    )
    embeddings = np.array(embeddings).astype("float32")
    
    # 构建索引
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    
    # 存储 (重置索引以匹配 FAISS 的 0-based ID)
    character_indices[char_name] = index
    character_dfs[char_name] = group.reset_index(drop=True)

print(f"\nCompleted! Built indices for {len(character_indices)} characters.")

Processing character: 凉宫春日 (Count: 933)


Batches: 100%|██████████| 30/30 [00:06<00:00,  4.49it/s]



Processing character: 沐雪 (Count: 2206)


Batches: 100%|██████████| 69/69 [00:07<00:00,  9.00it/s]



Processing character: 神里绫华 (Count: 1358)


...

Processing character: 胡桃 (Count: 816)


Batches: 100%|██████████| 26/26 [00:06<00:00,  3.97it/s]


Processing character: 钟离 (Count: 473)


Batches: 100%|██████████| 15/15 [00:04<00:00,  3.63it/s]


Completed! Built indices for 5 characters.


## Save Index and Mapping

In [7]:
SAVE_DIR = Path("./outputs/rag/")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

for char_name, index in character_indices.items():
    # 文件名安全处理
    safe_name = "".join([c for c in char_name if c.isalnum() or c in (' ', '_', '-')]).strip()
    safe_name = {"沐雪": "Muice", "神里绫华": "Ayaka", "钟离": "Zhongli", "胡桃": "Hutao", "凉宫春日": "Haruhi"}.get(char_name, safe_name)
    index_path = SAVE_DIR / f"{safe_name}_index.faiss"
    corpus_path = SAVE_DIR / f"{safe_name}_corpus.jsonl"
    
    faiss.write_index(index, str(index_path))
    character_dfs[char_name].to_json(str(corpus_path), orient="records", lines=True, force_ascii=False)
    
    print(f"Saved {char_name} -> {index_path}")

Saved 凉宫春日 -> outputs\rag\Haruhi_index.faiss
Saved 沐雪 -> outputs\rag\Muice_index.faiss
Saved 神里绫华 -> outputs\rag\Ayaka_index.faiss
Saved 胡桃 -> outputs\rag\Hutao_index.faiss
Saved 钟离 -> outputs\rag\Zhongli_index.faiss


## Demo: Retrieval Test

In [18]:
# 单句检索 (指定角色)

def search(character, query, top_k=5):
    if character not in character_indices:
        print(f"Error: Character '{character}' not found in indices.")
        return None

    index = character_indices[character]
    df = character_dfs[character]
    
    q_emb = embedder.encode([query], normalize_embeddings=True)
    scores, indices = index.search(np.array(q_emb).astype("float32"), top_k)
    
    results = df.iloc[indices[0]].copy()
    results["score"] = scores[0]
    return results

# 测试
test_char = "凉宫春日"
query = "你说的没错，但我不会因为别人的眼光限制自己的行动。我只关注自己想做的事，不在意他人的看法。"

print(f"🔍 查询角色: {test_char}")
print(f"🔍 查询内容: {query}\n")

results = search(test_char, query, top_k=5)
if results is not None:
    display(results)

🔍 查询角色: 凉宫春日
🔍 查询内容: 你说的没错，但我不会因为别人的眼光限制自己的行动。我只关注自己想做的事，不在意他人的看法。



,character,text,score
4,凉宫春日,哼，你这么说也没错。不过，我可不会因为别人的眼光而束缚自己的行动。我只在乎我自己想做的事情，...,0.933444
107,凉宫春日,阿虚，你太过紧张了。我们只是在享受自己的时间，做自己喜欢的事情。不要被别人的意见所束缚，相信...,0.597935
197,凉宫春日,我才不需要听你的意见呢，我自己的想法最重要！,0.595343
578,凉宫春日,哼，你自己决定吧。我可不会为你的感情问题操心。,0.584551
38,凉宫春日,自然就是我按照自己的方式行事，不受任何束缚和限制。我不喜欢被规则和常规所约束，我喜欢追求自己...,0.572016
